# KUKA IIWA-14 Controller

**Importing Required Libraries**

In [3]:
#setup nice plotting
%matplotlib widget
import time
import numpy as np
import pinocchio
import os
import meshcat
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy
from utils import Trajectory, move_joints_position, move_trajectory
np.set_printoptions(suppress=True, precision=4) # we don't want to print every decimal!

**Setting up environment and simulation**

In [4]:
package_dirs = './urdf/'
urdf_file = 'iiwa.urdf'
END_EFF_FRAME_ID = 17 # number of the frame corresponding to the end-effector
urdf = package_dirs + urdf_file
robot = pinocchio.RobotWrapper.BuildFromURDF(urdf, package_dirs)

viz = pinocchio.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
viz.loadViewerModel()
q = pinocchio.neutral(robot.model)
viz.display(q) 

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [11]:
from_q = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1])
to_q = 15*np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1])

trajectory = Trajectory(from_q=from_q, to_q=to_q, total_time=2.0)
move_trajectory(viz,frequency=100.0,trajectory=trajectory)

trajectory = Trajectory(from_q=to_q, to_q=from_q, total_time=2.0)
move_trajectory(viz,frequency=100.0,trajectory=trajectory)

In [15]:
viz.viewer['ball'].set_object(meshcat.geometry.Sphere(0.065), 
                              meshcat.geometry.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
des_pos = np.array([0.5, 0.5, 0.5])
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(des_pos))